In [67]:
import pandas as pd  
import numpy as np  
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC, SVR
from sklearn.model_selection import GridSearchCV

In [68]:
df = pd.read_csv("2019_done.csv")
indexed_dataSet, lookupTable = pd.factorize(df["Closest Location"].unique())
location_dict = {}

for x in range(0, indexed_dataSet.size):
    location_dict[lookupTable[x]] = indexed_dataSet[x]

In [69]:
from datetime import date
from datetime import datetime, timedelta

dt = datetime.combine(datetime.today(), datetime.min.time())
dt = dt - timedelta(weeks=1)
timestamp = datetime.timestamp(dt)

predict_df = pd.DataFrame()
predict_df["Closest Location"] = lookupTable
predict_df["Date"] = timestamp
predict_df.head()

precipIntensity = []
precipIntensityMax = []
precipIntensityMaxTime = []
temperatureHigh = []
temperatureHighTime = []
temperatureLow = []
temperatureLowTime = []
humidity = []

In [70]:
import requests
import json

station_df = pd.read_csv("station-coordinates.csv")

for row in predict_df.iterrows():

    station = station_df[station_df["Location"] == row[1].values[0]]
    
    lat = float(station["Latitude"])
    long = float(station["Longtitude"])
    time = int(timestamp)

    url = "https://api.darksky.net/forecast/477bf5a88e50f201a476d20e506afa12/{},{},{}".format(lat, long, time)
    r = requests.get(url)

    data = json.loads(r.text)
    data = data['daily']['data'][0]

    precipIntensity.append(data['precipIntensity'])
    precipIntensityMax.append(data['precipIntensityMax'])
    precipIntensityMaxTime.append(data['precipIntensityMaxTime'])
    temperatureHigh.append(data['temperatureHigh'])
    temperatureHighTime.append(data['temperatureHighTime'])
    temperatureLow.append(data['temperatureLow'])
    temperatureLowTime.append(data['temperatureLowTime'])
    humidity.append(data['humidity'])

predict_df['precipIntensity'] = precipIntensity
predict_df['precipIntensityMax'] = precipIntensityMax
predict_df['precipIntensityMaxTime'] = precipIntensityMaxTime
predict_df['temperatureHigh'] = temperatureHigh
predict_df['temperatureHighTime'] = temperatureHighTime
predict_df['temperatureLow'] = temperatureLow
predict_df['temperatureLowTime'] = temperatureLowTime
predict_df['humidity'] = humidity

In [71]:
predict_df["Closest Location"] = indexed_dataSet
predict_df = predict_df.drop(columns=["Date", 'precipIntensityMaxTime', 'temperatureHighTime', 'temperatureLowTime'])

In [72]:
predict_df.head()

,Closest Location,precipIntensity,precipIntensityMax,temperatureHigh,temperatureLow,humidity
0,0,0.0022,0.0121,87.39,75.98,0.87
1,1,0.0021,0.0113,87.52,76.06,0.88
2,2,0.0022,0.0148,87.75,77.45,0.85
3,3,0.0029,0.0182,87.63,77.49,0.85
4,4,0.0017,0.0157,88.20,78.44,0.83


In [73]:
import joblib

filename = 'xg_boost.sav'
loaded_model = joblib.load(filename)
results = loaded_model.predict_proba(predict_df)

In [74]:
results

array([[0.7004085 , 0.29959148],
       [0.86310244, 0.13689755],
       [0.8405073 , 0.15949269],
       [0.8405073 , 0.15949269],
       [0.78315836, 0.21684165],
       [0.8405073 , 0.15949269],
       [0.8405073 , 0.15949269],
       [0.8405073 , 0.15949269],
       [0.78315836, 0.21684165],
       [0.86310244, 0.13689755],
       [0.88623524, 0.11376477],
       [0.8668693 , 0.13313074],
       [0.8169347 , 0.18306531],
       [0.8169347 , 0.18306531],
       [0.8668693 , 0.13313074],
       [0.8668693 , 0.13313074],
       [0.8668693 , 0.13313074],
       [0.88623524, 0.11376477],
       [0.77152526, 0.2284747 ],
       [0.8169347 , 0.18306531],
       [0.8169347 , 0.18306531],
       [0.8169347 , 0.18306531],
       [0.8169347 , 0.18306531],
       [0.8169347 , 0.18306531],
       [0.8169347 , 0.18306531],
       [0.8668693 , 0.13313074],
       [0.8169347 , 0.18306531],
       [0.8169347 , 0.18306531],
       [0.83393013, 0.16606987],
       [0.83393013, 0.16606987],
       [0.

In [75]:
results_df = pd.DataFrame(lookupTable, columns=["Location"])

chance_list = []
for x in results:
    chance_list.append(x[1] * 100)
    
results_df["Probability"] = chance_list

In [39]:
results_df["Probability"] = 100

In [51]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore
from firebase_admin import db

project_id = "denguelive-47836"

# Use a service account
cred = credentials.Certificate("denguelive-47836-firebase-adminsdk-nunp7-b60052d8fe.json")
firebase_admin.initialize_app(cred, {
    'databaseURL' : 'https://denguelive-47836.firebaseio.com/'
})

ValueError: The default Firebase app already exists. This means you called initialize_app() more than once without providing an app name as the second argument. In most cases you only need to call initialize_app() once. But if you do want to initialize multiple apps, pass a second argument to initialize_app() to give each app a unique name.

In [40]:
# for firebase (doesn't work?)
for row in results_df.iterrows():
    location = row[1][0]
    probability = row[1][1]
    doc_ref = db.collection(u'locations').document(location)
    doc_ref.set({
        u'location': location,
        u'probability': probability,
    })

AttributeError: module 'firebase_admin.db' has no attribute 'collection'

In [94]:
# generate dict to add
prob_dict = {}
for row in results_df.iterrows():
    location = row[1][0]
    probability = row[1][1]
    prob_dict[location] = {'location': location, 'probability': probability}

In [53]:
prob_dict

{'Admiralty': {'location': 'Admiralty', 'probability': 29.959148168563843},
 'Admiralty West': {'location': 'Admiralty West',
  'probability': 13.689754903316498},
 'Ang Mo Kio': {'location': 'Ang Mo Kio', 'probability': 15.949268639087677},
 'Botanic Garden': {'location': 'Botanic Garden',
  'probability': 15.949268639087677},
 'Buangkok': {'location': 'Buangkok', 'probability': 21.68416529893875},
 'Bukit Panjang': {'location': 'Bukit Panjang',
  'probability': 15.949268639087677},
 'Bukit Timah': {'location': 'Bukit Timah', 'probability': 15.949268639087677},
 'Buona Vista': {'location': 'Buona Vista', 'probability': 15.949268639087677},
 'Chai Chee': {'location': 'Chai Chee', 'probability': 21.68416529893875},
 'Choa Chu Kang (Central)': {'location': 'Choa Chu Kang (Central)',
  'probability': 13.689754903316498},
 'Choa Chu Kang (South)': {'location': 'Choa Chu Kang (South)',
  'probability': 11.376477032899857},
 'Clementi': {'location': 'Clementi', 'probability': 13.313074409961

In [95]:
# for realtime database
ref = db.reference('/')
# Add a new user under /users.
ref.child('cases').set(
    prob_dict)

In [92]:
# normalisation
prob_col = results_df["Probability"]
results_df_norm = results_df.copy()
results_df_norm["Probability"] = (prob_col - prob_col.min())/(prob_col.max() - prob_col.min()) * 100

# save to file
results_df_norm.to_csv('bigQueryNorm.csv', index=False)

prob_dict = {}
for row in results_df_norm.iterrows():
    location = row[1][0]
    probability = row[1][1]
    prob_dict[location] = {'location': location, 'probability': probability}
    
# for realtime database
ref = db.reference('/')
# Add a new user under /users.
ref.child('cases').set(
    prob_dict)

import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore
from firebase_admin import db
from google.cloud import bigquery

project_id = "denguelive-47836"

# Use a service account
cred = credentials.Certificate("denguelive-47836-firebase-adminsdk-nunp7-b60052d8fe.json")

client = bigquery.Client()
filename = 'bigQueryNorm.csv'
dataset_id = 'firebase_predictions_v2'
table_id = 'dCluster_csv'

dataset_ref = client.dataset(dataset_id)
table_ref = dataset_ref.table(table_id)
job_config = bigquery.LoadJobConfig()
job_config.write_disposition = "WRITE_TRUNCATE"
job_config.source_format = bigquery.SourceFormat.CSV
job_config.skip_leading_rows = 1
job_config.autodetect = True

with open(filename, "rb") as source_file:
    job = client.load_table_from_file(source_file, table_ref, job_config=job_config)

job.result()  # Waits for table load to complete.

print("Loaded {} rows into {}:{}.".format(job.output_rows, dataset_id, table_id))

Loaded 37 rows into firebase_predictions_v2:dCluster_csv.


In [88]:
results_df["Probability"]

0     29.959148
1     13.689755
2     15.949269
3     15.949269
4     21.684165
5     15.949269
6     15.949269
7     15.949269
8     21.684165
9     13.689755
10    11.376477
11    13.313074
12    18.306531
13    18.306531
14    13.313074
15    13.313074
16    13.313074
17    11.376477
18    22.847471
19    18.306531
20    18.306531
21    18.306531
22    18.306531
23    18.306531
24    18.306531
25    13.313074
26    18.306531
27    18.306531
28    16.606987
29    16.606987
30    16.606987
31     9.363437
32    10.088754
33    10.088754
34     6.611947
35     7.265396
36     5.095763
Name: Probability, dtype: float64

In [93]:
# read from file
import pandas as pd
results_df = pd.read_csv('bigQuery.csv')

In [76]:
# save to file
results_df.to_csv('bigQuery.csv', index=False)

In [61]:
# Manual setting for demo

# Admiralty
results_df["Probability"][0] = 100

# Admiralty West
results_df["Probability"][1] = 40

C:\Users\zarch\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\zarch\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [62]:
results_df

,Location,Probability
0,Admiralty,100.000000
1,Admiralty West,40.000000
2,Ang Mo Kio,15.949269
3,Botanic Garden,15.949269
4,Buangkok,21.684165
5,Bukit Panjang,15.949269
6,Bukit Timah,15.949269
7,Buona Vista,15.949269
8,Chai Chee,21.684165
9,Choa Chu Kang (Central),13.689755


In [1]:
import os
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore
from firebase_admin import db
from google.cloud import bigquery

project_id = "denguelive-47836"

# Use a service account
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "denguelive-47836-firebase-adminsdk-nunp7-b60052d8fe.json"
cred = credentials.Certificate("denguelive-47836-firebase-adminsdk-nunp7-b60052d8fe.json")

client = bigquery.Client()
filename = 'bigQuery.csv'
dataset_id = 'firebase_predictions_v2'
table_id = 'dCluster_csv'

dataset_ref = client.dataset(dataset_id)
table_ref = dataset_ref.table(table_id)
job_config = bigquery.LoadJobConfig()
job_config.write_disposition = "WRITE_TRUNCATE"
job_config.source_format = bigquery.SourceFormat.CSV
job_config.skip_leading_rows = 1
job_config.autodetect = True

with open(filename, "rb") as source_file:
    job = client.load_table_from_file(source_file, table_ref, job_config=job_config)

job.result()  # Waits for table load to complete.

print("Loaded {} rows into {}:{}.".format(job.output_rows, dataset_id, table_id))

DefaultCredentialsError: Could not automatically determine credentials. Please set GOOGLE_APPLICATION_CREDENTIALS or explicitly create credentials and re-run the application. For more information, please see https://cloud.google.com/docs/authentication/getting-started